In [1]:
import pandas as pd
import numpy as np
import altair as alt
import os
import chromatose as ct

import geopandas as gpd
from geopy.distance import geodesic
from itertools import product
import json

from IPython.display import SVG
import scipy.special


/Users/rf50/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### Research question 1: How did the Palisades & Altadena fires of 2025 affect housing prices across Southern CA counties?

We model wildfires as the regional shock. Neighborhoods and counties close to the wildfires will experience housing price decline, but certain counties may see price increases due to spillover demand.

Potential counterfactual: 
“What would housing prices have been if 2020 wildfire intensity had been 50% lower?”


### Research question 2: How does the migration in response to housing price affect wages & labor allocation

Wildfires induce net outmigration in affected counties and net inmigration to safer counties, affecting demographics.

Potential counterfactual: 
“If wildfires were reduced by half, what would be the new distribution of labor and wages across counties?”

### 3. Research Question: How do wildfires influence migration patterns?
Potential counterfactual: If wildfire had occurred in a different location, how would migration flows have changed?


Datasets
- Zillow for regional houses
- Cal Fire: burn perimeters, burn intensity
- US Census / ACS: population, income, migration flows
- BLS / Quarterly Census of Employment and Wages (QCEW): county-level employment and wages
- LEHD / LODES: commuting flows




Moran's scatterplot for counties changes

How did the 2025 Eaton–Palisades wildfire affect the spatial allocation of housing, labor, and commuting across nearby California counties, and how did commuting linkages mitigate local economic losses?


### Read in and clean dataset column names

In [2]:
# filepath = 'spatial_econ_pset_1/datasets'
# # df_commuting = pd.read_csv(os.path.join(filepath, 'Commuting_Flows.csv'))

# df_housing = pd.read_csv(os.path.join(filepath, "socal_housing.csv"))
# df_pop = pd.read_csv(os.path.join(filepath, "socal_population.csv"))
# df_wages = pd.read_csv(os.path.join(filepath, "socal_empl_wages.csv"))
# df_commuting = pd.read_csv(os.path.join(filepath, 'socal_commuting_flows.csv'))

In [3]:
# df_pop = df_pop.rename(columns={
#     'County':'county',
#     'Total Population 24':'total_population_2024', 
#     'Total Population 25':'total_population_2025',
#     'Percent Change':'percent_change'
# })


# df_housing = df_housing.rename(columns={
#     'County':'county',
#     'Total Housing 24':'total_housing_2024', 
#     'Total Housing 25':'total_housing_2025',
#     'Percent Change':'percent_change'
# })


# df_wages = df_wages.rename(columns={
#     'County':'county',
#     'Num_Establishments':'n_establishments',
#     'Employment':'employment',
#     'Annual_Wages':'wages_annual',
#     'Weekly_Wage':'wages_weekly', 
#     'Wage_per_Employee':'wage_per_employee'
# })

# df_commuting = df_commuting.rename(columns={
#     'State Name':'state',
#     'Residence_County':'county_residence',
#     'Workplace_County':'county_workplace',
#     'Workers in Commuting Flow':'n_workers',
#     'Margin of Error': 'err_workers'
# })

# df_commuting['n_workers'] = df_commuting['n_workers'].str.replace(',','').astype(int)

- let's make a map with all the buildings of 2025 in the LA region
- then show which buildings burned in eaton fire

In [4]:
palette = ct.palpolate(ct.nacre[:-1])[::-1]

heatmap = (
    alt.Chart(df_commuting)
    .mark_rect()
    .encode(
        x=alt.X("county_workplace:N", title="Work county", axis=alt.Axis(labelAngle=45)),
        y=alt.Y("county_residence:N", title="Resident county"),
        color=alt.Color(
            "n_workers:Q",
            title="Number of workers",
            scale=alt.Scale(
                type="symlog", 
                range=palette)  # important for skewed flows
        ),
        tooltip=[
            "county_residence:N",
            "county_workplace:N",
            "n_workers:Q"
        ]
    )
    .properties(
        width=300,
        height=300,
        title="Commuting flows: residence → workplace"
    )
)

outside_share = (
    df_commuting
    .assign(outside=lambda d: d["county_workplace"] != d["county_residence"])
    .groupby("county_residence")
    .apply(lambda g: g.loc[g.outside, "n_workers"].sum() / g["n_workers"].sum())
    .reset_index(name="pct_outside")
)

outside_share["label"] = (outside_share["pct_outside"] * 100).round(1).astype(str) + "%"

text = (
    alt.Chart(outside_share)
    .mark_text(align="left", dx=5)
    .encode(
        y=alt.Y("county_residence:N"),
        x=alt.value(400),   # slightly to the right of heatmap width
        text="label:N"
    )
) 

# -----------------------------
outside_share_cell = (
    df_commuting
    .assign(outside=lambda d: d["county_workplace"] != d["county_residence"])
    .groupby("county_residence")
    .apply(lambda g: g.loc[g.outside, "n_workers"].sum() / g["n_workers"].sum())
    .reset_index(name="pct_outside")
)

outside_share_cell["label"] = (outside_share_cell["pct_outside"] * 100).round(1).astype(str) + "%"

text_2 = (
    alt.Chart(df_commuting)
    .transform_lookup(
        lookup="county_residence",
        from_=alt.LookupData(
            outside_share_cell,
            key="county_residence",
            fields=["label"]
        )
    )
    .mark_text(color="black", fontSize=9)
    .encode(
        x="county_workplace:N",
        y="county_residence:N",
        text="label:N"
    )
)
heatmap + text

/var/folders/6y/m69ngqb568nbb5t5p3lz216r0000gn/T/ipykernel_61512/653192324.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_commuting
/var/folders/6y/m69ngqb568nbb5t5p3lz216r0000gn/T/ipykernel_61512/653192324.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_commuting


alt.LayerChart(...)

In [5]:
outside_share

,county_residence,pct_outside,label
0,Los Angeles,0.065951,6.6%
1,Orange,0.143511,14.4%
2,Riverside,0.286024,28.6%
3,San Bernardino,0.277259,27.7%
4,San Diego,0.016521,1.7%
5,Ventura,0.174181,17.4%


### Construct modeling dataframes $d_{ij}$, $d_{i}$, $d_{j}$

In [6]:
# counties = df_pop['county'].values
# d_index_county = {i: county for i, county in enumerate(counties)}
# d_county_index = {v:k for k, v in d_index_county.items()}

# df_model = pd.DataFrame({
#     'county_i_name':counties, 
#     'county_j_name':counties
# })

# df_model['county_i'] = df_model['county_i_name'].map(d_county_index)
# df_model['county_j'] = df_model['county_j_name'].map(d_county_index)


In [7]:
# # ----------------- ij ----------------------
# df_model_ij = df_commuting.copy()
# df_model_ij['county_i_name'] = df_model_ij['county_residence']
# df_model_ij['county_j_name'] = df_model_ij['county_workplace']
# df_model_ij['L_ij_data'] = df_model_ij['n_workers']

# df_model_ij['county_i'] = df_model_ij['county_i_name'].map(d_county_index)
# df_model_ij['county_j'] = df_model_ij['county_j_name'].map(d_county_index)

# df_model_ij = df_model_ij[[
#     'county_i', 'county_j','county_i_name', 'county_j_name', 'L_ij_data']]

# # ----------------- j ----------------------
# df_model_j = df_wages.copy()
# df_model_j['county_j_name'] = df_model_j['county']
# df_model_j['county_j'] = df_model_j['county_j_name'].map(d_county_index)
# df_model_j['w_j_emp_data'] = df_model_j['wage_per_employee']
# df_model_j['w_j_county_data'] = df_model_j['wages_annual']
# df_model_j['L_j_data'] = df_model_j['employment']
# df_model_j['_n_firms_j_data'] = df_model_j['n_establishments']

# df_model_j = df_model_j[[
#     'county_j_name', 'county_j', 
#     'w_j_emp_data', 'w_j_county_data', 
#     'L_j_data', '_n_firms_j_data']]

# # ----------------- i ----------------------
# df_model_i = df_housing.merge(df_pop, on='county')
# df_model_i['county_i_name'] = df_model_i['county']
# df_model_i['county_i'] = df_model_i['county_i_name'].map(d_county_index)
# df_model_i['H_i_2024_data'] = df_model_i['total_housing_2024']
# df_model_i['H_i_2025_data'] = df_model_i['total_housing_2025']
# df_model_i['N_i_2024_data'] = df_model_i['total_population_2024']
# df_model_i['N_i_2025_data'] = df_model_i['total_population_2025']
# df_model_i = df_model_i[[
#     'county_i_name', 'county_i', 
#     'H_i_2024_data', 'H_i_2025_data', 'N_i_2024_data', 'N_i_2025_data']]

In [36]:
# df_model_ij.head(3)

In [37]:
# df_model_i.head(3)

In [38]:
# df_model_j.head(3)

Define spatial centroids of counties ['Los Angeles', 'Orange', 'Riverside', 'San Bernardino', 'San Diego', 'Ventura'] by population weighted weights.

### Get county centroids for $d_{ij}$

In [11]:
# d_county_fips = {
#     'Los Angeles':'037',
#     'Orange':'059',
#     'Riverside':'065',
#     'San Bernardino':'071',
#     'San Diego':'073',
#     'Ventura':'111',
# }
# d_fips_county = {v: k for k, v in d_county_fips.items()}

In [12]:
# KEEP THIS CELL - It's used for conversion (perhaps move to data cleaning)
# # --------- RETRIEVING .GEOJSON FROM .SHP -------
# shapefiles = [
#     'tl_2025_06_tract/tl_2025_06_tract.shp',
#     "tl_2025_us_county/tl_2025_us_county.shp",
#     "tl_2025_06_tract_simplify/tl_2025_06_tract.shp",
#     'Palisades_Perimeter_20250121_simplify/Palisades_Perimeter_20250121.shp',
#     'Eaton_Perimeter_20250121_simplify/Eaton_Perimeter_20250121.shp'
# ]
# name_shapefiles = [
#     "california_tracts_2020.geojson",
#     "california_counties_2020.geojson",
#     "california_tracts_2020_simplify.geojson",
#     'palisades_fire_perimeter.geojson',
#     'eaton_fire_perimeter.geojson',
# ]
# for shapefile_path, name_shapefile in zip(shapefiles, name_shapefiles):
#     gdf = gpd.read_file(shapefile_path)
#     print(gdf.crs)
    
#     # Export to GeoJSON
#     geojson_path = name_shapefile
#     gdf.to_file(geojson_path, driver="GeoJSON")
    
#     print("Saved GeoJSON:", geojson_path)

In [13]:
# used for centroid calculation, simplify used for plot
state_tracts = gpd.read_file(
    'california_tracts_2020.geojson')

state_tracts_simplify = gpd.read_file(
    'california_tracts_2020_simplify.geojson')
state_tracts_simplify = state_tracts_simplify.to_crs(epsg=4326)

state_counties = gpd.read_file('california_counties_2020.geojson')
state_counties = state_counties.to_crs(epsg=4326)

pali_fire = gpd.read_file('palisades_fire_perimeter.geojson')
eaton_fire = gpd.read_file('eaton_fire_perimeter.geojson')
pali_fire = pali_fire.set_crs(epsg=3857, allow_override=True).to_crs(epsg=4326)
eaton_fire = eaton_fire.set_crs(epsg=3857, allow_override=True).to_crs(epsg=4326)

In [14]:
# # Check bounds after projection
# print(pali_fire.total_bounds)  # [minx, miny, maxx, maxy] in degrees
# print(eaton_fire.total_bounds)

In [15]:
# # ---------------------
# # Compute centroids from state_tracts
# rows = []

# projected_crs = "EPSG:3310"
# state_tracts = state_tracts.to_crs(projected_crs)

# for k, v in d_county_fips.items():
#     county_tracts = state_tracts[state_tracts['COUNTYFP'] == v].copy()

#     # FIX: use county_tracts, not tracts
#     centroids = county_tracts.geometry.centroid

#     # robust mean centroid
#     mean_centroid = centroids.unary_union.centroid

#     rows.append({
#         "county_name": k,
#         "fips": v,
#         "centroid_x": mean_centroid.x,
#         "centroid_y": mean_centroid.y
#     })

# df_centroid = pd.DataFrame(rows)
# # ---------------
# gdf_centroid = gpd.GeoDataFrame(
#     df_centroid,
#     geometry=gpd.points_from_xy(df_centroid.centroid_x, df_centroid.centroid_y),
#     crs="EPSG:3310"
# )

# gdf_centroid = gdf_centroid.to_crs(epsg=4326)

# gdf_centroid["lon"] = gdf_centroid.geometry.x
# gdf_centroid["lat"] = gdf_centroid.geometry.y
# # ---------------

### Construct maps

In [39]:
# # County borders
# counties_json = json.loads(state_counties.to_json())
# county_chart = alt.Chart(alt.Data(values=counties_json['features'])).mark_geoshape(
#     fill='#eaeaea',
#     stroke='black',
# ).encode(
#     tooltip=[alt.Tooltip('properties.FID:Q', title='FID')]
# )

# # Centroid dots
# centroid_chart = alt.Chart(gdf_centroid).mark_circle(size=120, color="red").encode(
#     longitude="lon:Q",
#     latitude="lat:Q",
#     tooltip=["county_name"]
# )

In [40]:
# # Centroid dots + county borders
# map_centroid = (county_chart + centroid_chart).project(
#     type="mercator",
#     center=[-118, 34],
#     scale=2200
# ).properties(title='Approximate population-weighted centroids')

# map_centroid

Approximate bc we are taking the mean x and y of all census tract centroids. This assumes they are of roughly equal population.

In [18]:
# in full country x county dataset 
# FID: 963, 1993, 817, 362, 2943, 1896

In [41]:
# # --------------- Adding census tracts
# tracts_json = json.loads(state_tracts_simplify.to_json())
# tract_chart = alt.Chart(alt.Data(values=tracts_json['features'])).mark_geoshape(
#     fill=None,
#     stroke='#898989',
#     strokeWidth=0.25
# )

# # County borders + tract borders
# map_tract = (county_chart + tract_chart).project(
#     type="mercator",
#     center=[-118, 34],
#     scale=2800
# )
# # Centroid dots + county borders + tract borders
# map_tract_centroid = (county_chart + tract_chart + centroid_chart).project(
#     type="mercator",
#     center=[-118, 34],
#     scale=2800
# ).properties(title='Approximate population-weighted centroids')

# map_tract_centroid

In [42]:
# # --------------- Adding fire boundaries
# fill_fire = '#893434'
# stroke_fire = '#782323'
# pali_json = json.loads(pali_fire.to_json())
# pali_chart = alt.Chart(alt.Data(values=pali_json['features'])).mark_geoshape(
#     fill=fill_fire, stroke=stroke_fire,
#     strokeWidth=0.25, opacity=0.9
# )
# eaton_json = json.loads(eaton_fire.to_json())
# eaton_chart = alt.Chart(alt.Data(values=eaton_json['features'])).mark_geoshape(
#     fill=fill_fire, stroke=stroke_fire,
#     strokeWidth=0.25, opacity=0.9
# )

# map_fire_tract = (
#     county_chart + tract_chart + pali_chart + eaton_chart
# ).project(
#     type="mercator",
#     center=[-118, 34],
#     scale=7000
# ).properties(title=['Eaton & Palisades fire perimeters', 'Overlaid with census tracts'])

# map_fire_centroid_tract = (
#     county_chart + tract_chart + pali_chart + eaton_chart + centroid_chart
# ).project(
#     type="mercator",
#     center=[-118, 34],
#     scale=2800
# ).properties(title="Surrounding counties of LA")

In [43]:
# alt.renderers.set_embed_options(actions=False, renderer='svg')

# map_fire_tract.save('map_fire_tract.svg')
# map_fire_centroid_tract.save('map_fire_centroid_tract.svg')

In [44]:
# SVG(filename='map_fire_tract.svg')

In [45]:
# SVG(filename='map_fire_centroid_tract.svg')

### Construct $d_{ij}$

In [22]:
pairs = pd.DataFrame(
    list(product(df_centroid.fips, df_centroid.fips)
), columns=["fips_origin", "fips_dest"])

# Merge coordinates for county A
pairs = pairs.merge(df_centroid[['fips','centroid_x','centroid_y']], left_on='fips_origin', right_on='fips')
pairs.rename(columns={'centroid_x':'x_origin','centroid_y':'y_origin'}, inplace=True)
pairs.drop(columns='fips', inplace=True)

# Merge coordinates for county B
pairs = pairs.merge(df_centroid[['fips','centroid_x','centroid_y']], left_on='fips_dest', right_on='fips')
pairs.rename(columns={'centroid_x':'x_dest','centroid_y':'y_dest'}, inplace=True)
pairs.drop(columns='fips', inplace=True)

# Euclidean distance in meters
pairs['distance_m'] = np.sqrt((pairs['x_dest'] - pairs['x_origin'])**2 + (pairs['y_dest'] - pairs['y_origin'])**2)
pairs['distance_mi'] = pairs['distance_m'] / 1609.34
pairs['d_ij'] = pairs['distance_mi']

pairs['county_i_name'] = pairs['fips_origin'].map(d_fips_county)
pairs['county_j_name'] = pairs['fips_dest'].map(d_fips_county)

pairs = pairs[['county_i_name', 'county_j_name', 'd_ij']]
pairs.head()

,county_i_name,county_j_name,d_ij
0,Los Angeles,Los Angeles,0.000000
1,Los Angeles,Orange,32.571940
2,Los Angeles,Riverside,74.016584
3,Los Angeles,San Bernardino,54.615943
4,Los Angeles,San Diego,105.441086


In [23]:
# using web-widget here, we can estimate the diagonals (within county commute LA-LA commute)
# https://www.census.gov/acs/www/about/why-we-ask-each-question/commuting/
# We can then multiply by some scalar factor
# LA: 30 min commute
# OC: 27 min commute
# San Bernardino: 33 minute commute
# Ventura: 26 min commute
# Riverside: 34 min commute
# San Diego: 26 min commute

# in minutes (median from ACS census widget)
factor = 0.25
d_inter_county_commute = {
    'Los Angeles': 30,
    'Orange': 27,
    'San Bernardino': 33,
    'San Diego': 26,
    'Riverside': 34,
    'Ventua': 26
}

# min per mile (factors suggested by chatgpt)
d_inter_county_commute_factor = {
    'Los Angeles': 2.3,
    'Orange': 2.0,
    'San Bernardino': 1.6,
    'San Diego': 2.0,
    'Riverside': 1.6,
    'Ventua': 1.7
}

for k, v in d_inter_county_commute.items():
    factor = d_inter_county_commute_factor[k]
    pairs.loc[
        (pairs['county_i_name']==k) & 
        (pairs['county_j_name']==k)
    , 'd_ij'] = v / factor

In [24]:
pairs.head(3)

,county_i_name,county_j_name,d_ij
0,Los Angeles,Los Angeles,13.043478
1,Los Angeles,Orange,32.571940
2,Los Angeles,Riverside,74.016584


In [25]:
df_model_ij = pd.merge(
    df_model_ij, pairs, how='left', on=['county_i_name','county_j_name'])

In [211]:
df_model_ij.head()

,county_i,county_j,county_i_name,county_j_name,L_ij_data,d_ij
0,0,0,Los Angeles,Los Angeles,4429523,13.043478
1,1,0,Orange,Los Angeles,180250,32.571940
2,2,0,Riverside,Los Angeles,53172,74.016584
3,3,0,San Bernardino,Los Angeles,132992,54.615943
4,4,0,San Diego,Los Angeles,6075,105.441086


### Add.l Parameters: $F_i$

F: In estimating wildfire exposure of LA, we think about the approximate proportion of structures that were destroyted by the wildfires. If we make the assumption that 10,000 of 1 million structures were destroyed by the fires:
\begin{align}
&F_i = 0.01 \hspace{0.3em} i = \mathrm{Los Angeles}\\
&F_i = 0.00 \hspace{0.3em} \mathrm{otherwise} \\
\end{align}

In [26]:
d_Fi_nofire = {
    'Los Angeles':0, 
    'Orange':0, 
    'Riverside':0, 
    'San Bernardino':0,
    'San Diego':0, 
    'Ventura':0
}
d_Fi_fire1 = {
    'Los Angeles':0.01, 
    'Orange':0, 
    'Riverside':0, 
    'San Bernardino':0,
    'San Diego':0, 
    'Ventura':0
}
df_model_i['F_i_nofire'] = df_model_i['county_i_name'].map(d_Fi_nofire)
df_model_i['F_i_fire1'] = df_model_i['county_i_name'].map(d_Fi_fire1)

In [27]:
kappa = 0.08 # redding 2016: 0.05–0.15 per mile, 3-10% wage loss per 10 miles
chi = 0.5 # calibrate this: low chi: small response, high chi: large response
eta = 0.7 # housing supply elasticity saiz 2010

We can come back to estimate $\kappa$ from the gravity eqns, but for now we will set it to 0.08 and move on
\begin{align}
L_{ij}^{data} = \pi_{ij} N
\end{align}

We have now built $d_{ij}$, $F_i$, $Z_j$ from wages.   
We will solve for baseline equilibrium.   
Then increase $F_{LA}$, and re-solve for equilibrium.   
Compare population, wages, housing, and commuting.

### Back out baseline housing prices

At spatial equilibrium:  
- labor markets clear
- housing markets clear
- commuting flows reflect choices
  households are indifferent across residence-workplace pairs

Housing market clearing:
\begin{align}
p_i = \left(\frac{L_i}{H_i}\right)^{1/\eta}
\end{align}

In [29]:
df_model_i['p_i_eq'] = (df_model_i['N_i_2024_data'] / df_model_i['H_i_2024_data'])**(1/eta)

In [30]:
df_model_i[['county_i_name', 'p_i_eq']].sort_values(by='p_i_eq', ascending=False)

,county_i_name,p_i_eq
3,San Bernardino,4.638719
2,Riverside,4.404391
5,Ventura,4.308436
1,Orange,4.217296
0,Los Angeles,4.055043
4,San Diego,3.952133


We notice that $p_i$ is roughly inverse from known median county home prices (SB low, OC LA and SD have very high prices).

### Normalize wages w_j for Z_j

In [31]:
# normalize weighted avg wage per employee
weights = df_model_j['L_j_data'] / df_model_j['L_j_data'].sum()
w_bar = np.sum(weights * df_model_j['w_j_emp_data']) # weighted avg
df_model_j['w_j_emp_norm_data'] = df_model_j['w_j_emp_data'] / w_bar

w_per_capita = df_model_j['w_j_county_data'] / df_model_j['L_j_data']
w_bar_capita = np.sum(weights * w_per_capita)
df_model_j['w_j_county_norm_data'] = w_per_capita / w_bar_capita

### Compute baseline $\pi_{ij}$

In [34]:
# merge ij, i, j dataframes for \pi_ij computation
df_model_merge = df_model_ij.copy()
df_model_merge = df_model_merge.merge(
    df_model_i, on='county_i_name'
    ).merge(df_model_j, on='county_j_name')

In [35]:
df_model_merge.columns

Index(['county_i_x', 'county_j_x', 'county_i_name', 'county_j_name',
       'L_ij_data', 'd_ij', 'county_i_y', 'H_i_2024_data', 'H_i_2025_data',
       'N_i_2024_data', 'N_i_2025_data', 'F_i_nofire', 'F_i_fire1', 'p_i_eq',
       'county_j_y', 'w_j_emp_data', 'w_j_county_data', 'L_j_data',
       '_n_firms_j_data', 'w_j_emp_norm_data', 'w_j_county_norm_data'],
      dtype='object')

In [224]:
df_model_merge['_U_ij_base_nofire'] = np.log(df_model_merge['w_j_per_employee_norm_data']) \
    - np.log(df_model_merge['p_i_eq']) \
    - kappa * df_model_merge['d_ij'] - chi * df_model_merge['F_i_nofire']

df_model_merge['pi_ij'] = scipy.special.softmax(df_model_merge['_U_ij_base_nofire'])

In [232]:
total_N = df_model_i['N_2024_data'].sum()

In [229]:
df_model_merge['pi_ij'] * total_

0     1.388299e+06
1     2.798699e+05
2     9.731279e+03
3     4.362195e+04
4     8.778599e+02
5     9.261052e+04
6     2.835750e+05
7     1.253881e+06
8     6.896366e+04
9     9.206173e+04
10    1.138658e+04
11    8.272139e+03
12    7.556395e+03
13    5.285101e+04
14    4.739398e+05
15    1.788277e+05
16    1.726088e+04
17    1.775835e+02
18    3.724793e+04
19    7.758263e+04
20    1.966469e+05
21    4.939306e+05
22    1.907184e+03
23    1.054064e+03
24    8.484134e+02
25    1.086085e+04
26    2.148328e+04
27    2.158628e+03
28    1.417437e+06
29    3.291988e+01
30    8.300920e+04
31    7.317657e+03
32    2.049858e+02
33    1.106460e+03
34    3.053105e+01
35    3.129518e+06
Name: pi_ij, dtype: float64

In [ ]:
pi_ij * L_total ≈ N_ij_data

We have now built $d_{ij}$, $F_i$, $Z_j$ from wages.   
We will solve for baseline equilibrium parameters.   
Then increase $F_{LA}$, and re-solve for equilibrium.
Compare population, wages, housing, and commuting.

At spatial equilibrium: 
- labor markets clear
- housing markets clear
- commuting flows reflect choices
  households are indifferent across residence-workplace pairs


In [ ]:
Given:
- distance matrix d_ij
- wildfire exposure F_i
- housing stock H_i
- productivity Z_j
- parameters (kappa, chi, eta)

Initialize:
- wages w_j = Z_j
- prices P_i = 1
- population N_i = N / I

Repeat until convergence:
  1. Compute utility U_ij
  2. Compute choice probabilities π_ij
  3. Compute commuting flows L_ij = N * π_ij
  4. Update:
       N_i = sum_j L_ij
       L_j = sum_i L_ij
  5. Update wages:
       w_j = Z_j
       (or w_j = MPL_j if you want diminishing returns)
  6. Update housing prices:
       P_i = (N_i / H_i)^η
End

Return equilibrium objects